<a href="https://colab.research.google.com/github/reymage/2021-pydantic/blob/main/indrive_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install google-play-scraper

In [7]:
from google_play_scraper import app, Sort, reviews_all, reviews
import pandas as pd
import numpy as np
# from app_store_scraper import AppStore
# import json, os, uuid

In [19]:
# Lt's scrape the data
play_id = "sinet.startup.inDriver"
# app_id = 780125801

# Define the app ID for inDrive
app_id = 'com.indriver.client'  # Replace with the actual app ID if different

# Set up variables for pagination and unique reviews
all_reviews = []
review_ids = set()
continuation_token = None
max_reviews_per_request = 200  # Number of reviews to fetch per request
total_reviews_to_fetch = 50000   # Total number of reviews to fetch
fetched_reviews_count = 0

while fetched_reviews_count < total_reviews_to_fetch:
    # Calculate how many reviews to fetch in this iteration
    remaining_reviews = total_reviews_to_fetch - fetched_reviews_count
    current_fetch_count = min(max_reviews_per_request, remaining_reviews)

    # Fetch reviews with pagination
    fetched_reviews, continuation_token = reviews(
        play_id,
        lang='en',  # Language of the reviews
        country='NG',  # Country code for Nigeria
        sort=Sort.NEWEST,  # Sort by newest reviews
        count=current_fetch_count,  # Number of reviews to fetch
        continuation_token=continuation_token  # Pagination token
    )

    for review in fetched_reviews:
        if review['reviewId'] not in review_ids:
            review_ids.add(review['reviewId'])  # Store unique review ID
            all_reviews.append(review)  # Add unique review to the list
            fetched_reviews_count += 1    # Increment count of fetched reviews

    # Break if no more reviews are available (no continuation token)
    if not continuation_token or fetched_reviews_count >= total_reviews_to_fetch:
        break
    if continuation_token and not fetched_reviews:  # I inserted this if statement myself.
        break  # Exit loop if no more reviews

# Convert the list of unique reviews to a DataFrame for easier manipulation
df_reviews = pd.DataFrame(all_reviews)

# Display the first few rows of the DataFrame
print(df_reviews.head())

# Optionally, save to a CSV file
df_reviews.to_csv('indrive_nigeria_reviews.csv', index=False)

                               reviewId                      userName  \
0  625a5a3e-8846-4320-9272-c197604c3623                        Ragu S   
1  243d2cf0-3bc5-489c-b8bf-5058241349c6                  Ezzataly Aly   
2  59b18252-5f9c-4105-b455-482442e58a4f  Roman Shcherbakov (Flameash)   
3  11401991-f35d-4f4f-a86e-20e4fed67d55                Muhammed Kabir   
4  1d072b4b-5717-4235-87d1-9ee91b907a32              Alejandro Loredo   

                                           userImage  \
0  https://play-lh.googleusercontent.com/a/ACg8oc...   
1  https://play-lh.googleusercontent.com/a/ACg8oc...   
2  https://play-lh.googleusercontent.com/a-/ALV-U...   
3  https://play-lh.googleusercontent.com/a/ACg8oc...   
4  https://play-lh.googleusercontent.com/a-/ALV-U...   

                                             content  score  thumbsUpCount  \
0  பயணி ஒருகுறிப்பிட்ட வாகனத்தை தேர்ந்தெடுக்கிற இ...      1              0   
1                                          Very well      5         

In [20]:
# find shape of data
df_reviews.shape

(50000, 11)